# Nada AI Inference Template

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NillionNetwork/nada-ai/blob/main/templates/customizable_inference/inference_template.ipynb)

This notebook provides a generic & easily customizable end-to-end template for running AI model inference on the Nillion network.

Feel free to customize this template to fit your use case by navigating to the cells annotated by a 📝 TODO symbol!

We are really excited for developers to build with our SDK, if you have any questions please do reach out to us on:

[![Discord](https://img.shields.io/badge/Discord-nillionnetwork-%235865F2?logo=discord)](https://discord.gg/nillionnetwork)
[![GitHub Discussions](https://img.shields.io/badge/GitHub_Discussions-NillionNetwork-%23181717?logo=github)](https://github.com/orgs/NillionNetwork/discussions)

# 1. Set up environment

The boring part!

Installs all required dependencies and spins up a local devnet that will run Nada programs

In [1]:
%%capture
!pip install nada-ai~=0.3.0 seaborn

In [2]:
import os
import time
import sys
import uuid
import torch
import seaborn as sns
import torch.optim as optim
import numpy as np
import nada_numpy as na
from nada_ai.client import TorchClient
import pandas as pd
import nada_numpy.client as na_client
from dotenv import load_dotenv
from nillion_python_helpers import create_nillion_client, create_payments_config, get_quote, get_quote_and_pay, pay_with_quote
import py_nillion_client as nillion
from py_nillion_client import NodeKey, UserKey
from cosmpy.aerial.client import LedgerClient
from cosmpy.aerial.wallet import LocalWallet
from cosmpy.crypto.keypairs import PrivateKey

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch import nn
from torch.utils.data import DataLoader, Dataset
from pathlib import Path
from IPython.core.magic import register_cell_magic

In [3]:
os.makedirs("target/", exist_ok=True)

In [4]:
@register_cell_magic
def to_file(line, cell):
    "Writes the content of the cell to a file specified in the line argument."
    filepath = Path(line.strip())
    filepath.parent.mkdir(parents=True, exist_ok=True)

    with open(filepath, "w") as f:
        f.write(cell)

In [5]:
# Configure telemetry settings
enable_telemetry = True  #@param {type:"boolean"}
my_identifier = "your-telemetry-identifier"  #@param {type:"string"}

In [6]:
# Install the nilup tool and then use that to install the Nillion SDK
!curl https://nilup.nilogy.xyz/install.sh | bash

# Update Path if ran in colab
if "google.colab" in sys.modules:
    os.environ["PATH"] += ":/root/.nilup/bin"
    os.environ["PATH"] += ":/root/.nilup/sdks/latest/"

# Set telemetry if opted in
if enable_telemetry:
    identifier = f"nada-ai-inference-{str(uuid.uuid4())}-{my_identifier}"
    !echo 'yes' | nilup instrumentation enable --wallet {identifier}

# Install the lastest SDK and initialise it
!nilup init
!nilup install 0.4.0
!nilup use 0.4.0

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7810  100  7810    0     0  11504      0 --:--:-- --:--:-- --:--:-- 11519

nilup has been installed into /root/.nilup/bin and added to your $PATH in /root/.bashrc.

Run 'source /root/.bashrc' or start a new terminal session to use nilup.

By providing your Ethereum wallet address, you consent to the collection of telemetry data by the Nillion Network.
That includes but is not limited to
- The version of the SDK you are using
- The OS you are using
- The Processor Architecture you are using
- The SDK binary that you are running and the subcommand
- The wallet address you provided
- The errors produced by the SDK
We collect this data to understand how the software is used, and to better assist you in case of issues.
While we will not collect any personal information, we still recommend using a new wallet address that cannot be 

In [7]:
# Spin up local Nillion devnet
!nohup nillion-devnet &

time.sleep(20)  # Wait for devnet

nohup: appending output to 'nohup.out'


# 2. Load dataset

We'll start of by loading the dataset we want to train our model on.

In this example, we use an AI classic: the Titanic dataset.

**TODO: 📝 replace this by your own dataset!**

In [8]:
# TODO: 📝 read your own dataset!
raw_df = sns.load_dataset("titanic")

train_df, test_df = train_test_split(
    raw_df,
    test_size=0.20,
    random_state=42,
)

Next, we will do some pre-processing & general data cleaning.

**TODO: 📝 (optional) replace this by your own pre-processing!**

In [9]:
# TODO: 📝 define your own preprocessing logic here
def preprocess_data(data: pd.DataFrame) -> pd.DataFrame:
    data = data.drop(columns=["deck", "embark_town", "alive", "class", "who", "adult_male", "alone"])
    data = data.rename({"survived": "label"}, axis=1)
    data["age"] = data["age"].fillna(data["age"].mean())
    data["embarked"] = data["embarked"].fillna(data["embarked"].mode()[0])
    data["embarked"] = data["embarked"].map({"S": 0, "C": 1, "Q": 2})
    data["sex"] = data["sex"].map({"male": 0, "female": 1})
    scaler = StandardScaler()
    X = data.drop("label", axis=1)
    y = data["label"]
    X = pd.DataFrame(scaler.fit_transform(X.values), columns=X.columns, index=X.index)
    data = pd.concat([X, y], axis=1)
    return data

train_df = preprocess_data(train_df)
test_df = preprocess_data(test_df)

Now, we will transform our dataset into a custom PyTorch Dataset.

Here, we assume that:
- The dataset consists of only numerical values.
- The dataset has a singular column called `label` that is the prediction target.

In [10]:
class MyDataset(Dataset):
    def __init__(
        self,
        data: pd.DataFrame,
        target: str="label",
    ) -> None:
        super().__init__()
        self.data = data
        self.features = self.data.drop(target, axis=1).values.astype(float)
        self.targets = self.data[target].values.astype(float)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        feature = torch.tensor(self.features[idx], dtype=torch.float32)
        target = torch.tensor(self.targets[idx], dtype=torch.float32)
        return feature, target

train_dataset = MyDataset(train_df)
test_dataset = MyDataset(test_df)

Finally, we wrap our newly created datasets in a PyTorch DataLoader for efficiency reasons.

In [11]:
train_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
)

test_loader = DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=False,
)

# 3. Create model

Now that we have our dataset ready, it's time to define which model we want to use.

**Important: we need to ensure that we pick a model architecture that is supported by `nada-ai` and model dimensions that work within the capacity of the network.**
You can find more information [here](https://docs.nillion.com/nada-ai-introduction#supported-models)

In this example, we will use a small neural net.

**TODO: 📝 make your own model or load a pre-trained one**

In [12]:
# TODO: 📝 make your own model or load a pre-trained one

class MyModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.ln1 = nn.Linear(7, 16)
        self.ln2 = nn.Linear(16, 1)
        self.act = nn.ReLU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.ln1(x)
        x = self.act(x)
        x = self.ln2(x)
        return x

my_model = MyModel()

# 4. Train model

Now that we have both a dataset and a model, the time has come to train our model on the dataset.

**Warning**: this example assumes that we are training a model to perform a single-class classification task.
If your task differs from this, you will likely need to modify the training process.

In [13]:
training_args = {
    "num_train_epochs": 100,
    "learning_rate": 0.01,
}

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(
    my_model.parameters(),
    lr=training_args["learning_rate"],
)

for epoch in range(training_args["num_train_epochs"]):
    my_model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = my_model(inputs).squeeze(1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    my_model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = torch.sigmoid(my_model(inputs))
            predicted = torch.round(outputs).squeeze(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    if (epoch) % 10 == 0:
        print(f'Epoch {epoch}, Loss: {running_loss/len(train_loader):.6f}')
        print(f'Validation Accuracy: {100 * correct / total:.4f}%')

Epoch 0, Loss: 0.526559
Validation Accuracy: 79.8883%
Epoch 10, Loss: 0.395110
Validation Accuracy: 79.3296%
Epoch 20, Loss: 0.390059
Validation Accuracy: 79.8883%
Epoch 30, Loss: 0.385221
Validation Accuracy: 78.2123%
Epoch 40, Loss: 0.378938
Validation Accuracy: 78.2123%
Epoch 50, Loss: 0.382115
Validation Accuracy: 78.7709%
Epoch 60, Loss: 0.371655
Validation Accuracy: 78.7709%
Epoch 70, Loss: 0.368409
Validation Accuracy: 78.7709%
Epoch 80, Loss: 0.363957
Validation Accuracy: 79.8883%
Epoch 90, Loss: 0.363746
Validation Accuracy: 79.3296%


# 5. Create Nada program that runs blind inference

At this point, we have a trained PyTorch model.

Now, we want to create a program that loads this model, loads some dataset and performs blind inference.

Luckily we have just the tool for that: it's called Nada!

We'll start off by writing a small .toml file that defines some basic configurations related to our project.

In [14]:
%%to_file nada-project.toml
name = "inference"
version = "0.1.0"
authors = [""]

[[programs]]
path = "src/inference.py"
prime_size = 128

Now we can implement a program that:
- Loads our brand new model, using `nada-ai`
- Loads the dataset we want to run blind inference on, using `nada-numpy`
- Runs blind inference
- Returns the inference result, using the `nada-dsl`

**TODO: 📝 implement your own model using nada-ai modules**

In [15]:
%%to_file src/inference.py
import nada_numpy as na

from nada_ai import nn
from nada_dsl import Output
from typing import List

INPUT_DIMS=(7,)  # we specify the dimensions of the input dataset

# TODO: 📝 implement your model using nada-ai modules
class MyModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.ln1 = nn.Linear(7, 16)
        self.ln2 = nn.Linear(16, 1)
        self.act = nn.ReLU()

    def forward(self, x: na.NadaArray) -> na.NadaArray:
        x = self.ln1(x)
        x = self.act(x)
        x = self.ln2(x)
        return x

def nada_main() -> List[Output]:
    """Main Nada program"""
    # There will be two parties in this example: a party that
    # provides the model and another party that provides a dataset
    parties = na.parties(2)

    # First, we randomly initialize the model we wish to load
    my_model = MyModel()

    # Next, we load the model weights as secrets from the network
    # We assume the model weights are floats provided by the first party
    # It will attempt to load a set of secrets named `my_model`
    my_model.load_state_from_network(
        "my_model",
        parties[0],
        na.SecretRational,
    )

    # Then, we load the dataset as secrets from the network
    # We assume the dataset are floats provided by the second party
    # It will attempt to load a set of secrets named `my_input`
    my_input = na.array(
        INPUT_DIMS,
        parties[1],
        "my_input",
        na.SecretRational,
    )

    # Finally, we can run blind inference by simply providing the
    # inference dataset to the dataset
    result = my_model(my_input)

    # We will return the result to the second party
    return result.output(parties[1], "my_output")

Now that we have our inference program, we can build it using the `nada` CLI tool

In [16]:
!nada build

Building program: inference
Build complete!


# 6. Deploy our Nada inference program

Now that we have a set of instructions we want to execute in MPC (aka a built Nada program), we can proceed to deploying this model on the network.

In [17]:
# We define some config variables

home = os.getenv("HOME")
load_dotenv(f"{home}/.config/nillion/nillion-devnet.env")

cluster_id = os.getenv("NILLION_CLUSTER_ID")
grpc_endpoint = os.getenv("NILLION_NILCHAIN_GRPC")
chain_id = os.getenv("NILLION_NILCHAIN_CHAIN_ID")
seed = "my_seed"

userkey = UserKey.from_seed((seed))
nodekey = NodeKey.from_seed((seed))

client = create_nillion_client(userkey, nodekey)
party_id = client.party_id
user_id = client.user_id

party_names = na_client.parties(2)
program_name = "inference"
program_mir_path = f"target/{program_name}.nada.bin"

payments_config = create_payments_config(chain_id, grpc_endpoint)
payments_client = LedgerClient(payments_config)
payments_wallet = LocalWallet(
    PrivateKey(bytes.fromhex(os.getenv("NILLION_NILCHAIN_PRIVATE_KEY_0"))),
    prefix="nillion",
)

In [18]:
# Next we pay and provide a payment receipt to deploy our Nada program

quote_store_program = await get_quote(
    client, nillion.Operation.store_program(program_mir_path), cluster_id
)

receipt_store_program = await pay_with_quote(
    quote_store_program, payments_wallet, payments_client
)

action_id = await client.store_program(
    cluster_id, program_name, program_mir_path, receipt_store_program
)

program_id = f"{user_id}/{program_name}"

print("Program deployed!")
print("program_id:", program_id)

Getting quote for operation...
Submitting payment receipt 2 unil, tx hash 9CFFFFA2FE7DEBB0F426EEE650CB2A6B92C7AB9FD467CF3EB4FE8C5FCF5738E2
Program deployed!
program_id: 3rgqxWd47e171EUwe4RXP9hm45tmoXfuF8fC52S7jcFoQTnU8wPiL7hqWzyV1muak6bEg7iWhudwg4t2pM9XnXcp/inference


# 7. Provide data

Now that we have a deployed program, the input parties can provide the data that the program requires in order to run correctly.

In our example, we need:
- The first party to provide the model weights.
- The second party to provide the inference data.

In [19]:
permissions = nillion.Permissions.default_for_user(client.user_id)
permissions.add_compute_permissions({client.user_id: {program_id}})

First, we'll let the first party supply the weights of the model we trained earlier.

In [20]:
# We wrap the model in the relevant nada-ai client for the AI/ML framework we are using (here: PyTorch)
model_client = TorchClient(my_model)

# Now we can export the model state as secrets and name those secrets `my_model`
model_secrets = nillion.NadaValues(
    model_client.export_state_as_secrets("my_model", na.SecretRational)
)

In [21]:
# Finally, we can pay and store the model weights in the network as secrets
receipt_store_model = await get_quote_and_pay(
    client,
    nillion.Operation.store_values(model_secrets, ttl_days=1),
    payments_wallet,
    payments_client,
    cluster_id,
)

model_store_id = await client.store_values(cluster_id, model_secrets, permissions, receipt_store_model)

Getting quote for operation...
Quote cost is 13922 unil
Submitting payment receipt 13922 unil, tx hash 6FCE3967A2C819B8539185A1B542EDD3378AC7955EAFB7E0AF88F2CA2B37A5FE


Next, we'll let the second party supply some input dataset

**TODO: 📝 provide your own input data**

In [22]:
# TODO: 📝 provide your own input data
input_data = test_df.iloc[0].to_numpy()

# We can export the data array as secrets and name those secrets `my_input`
my_input = na_client.array(input_data, "my_input", na.SecretRational)
input_secrets = nillion.NadaValues(my_input)

In [23]:
# Finally, we can pay and store the input data in the network as secrets
receipt_store_data = await get_quote_and_pay(
    client,
    nillion.Operation.store_values(input_secrets, ttl_days=1),
    payments_wallet,
    payments_client,
    cluster_id,
)

data_store_id = await client.store_values(cluster_id, input_secrets, permissions, receipt_store_data)

Getting quote for operation...
Quote cost is 770 unil
Submitting payment receipt 770 unil, tx hash F379E971717549B153E1B8EFAAEC0F65C229B00EFEA74124430BD2DEF4A6FF7C


# 8. Run inference

Finally, everything we wanted is in place:
- We created & deployed a Nada program that loads a model named `my_model` and a dataset named `my_input`, runs inference and returns the result.
- We trained a model and uploaded the weights to the network as `my_model`.
- We defined a dataset we'd like to run inference on and uploaded it to the network as `my_input`.

So, without further ado, we can actually execute the program now.

In [24]:
compute_bindings = nillion.ProgramBindings(program_id)

for party_name in party_names:
    compute_bindings.add_input_party(party_name, party_id)

compute_bindings.add_output_party(party_names[-1], party_id)

In [25]:
computation_time_secrets = nillion.NadaValues({})

receipt_compute = await get_quote_and_pay(
    client,
    nillion.Operation.compute(program_id, computation_time_secrets),
    payments_wallet,
    payments_client,
    cluster_id,
)

_ = await client.compute(
    cluster_id,
    compute_bindings,
    [model_store_id, data_store_id],
    computation_time_secrets,
    receipt_compute,
)

while True:
    compute_event = await client.next_compute_event()
    if isinstance(compute_event, nillion.ComputeFinishedEvent):
        print(f"✅ Compute complete for compute_id {compute_event.uuid}")
        result = compute_event.result.value
        break

result = {
    key: na_client.float_from_rational(value)
    for key, value in result.items()
}
print("Result is", result)

Getting quote for operation...
Quote cost is 388 unil
Submitting payment receipt 388 unil, tx hash 02AFC87804D8827CEC66D78A632DF4DE179DDB8CEB344BF53ACE5AEC0E3AAECF
✅ Compute complete for compute_id 6e7658b5-ba68-437e-9d90-4fc3008e00ea
Result is {'my_output_0': -2.356842041015625}
